In [132]:
import gc
import json
import numpy as np
import pandas as pd
import seaborn as sns
from pandas.io.json import json_normalize
from tqdm import tqdm
from matplotlib import pyplot as plt
from functools import partial
from sklearn.preprocessing import LabelEncoder
plt.style.use('fivethirtyeight')

In [2]:
features = [
    'channelGrouping', 'date', 'fullVisitorId', 'visitId', 'visitNumber', 'visitStartTime', 'customDimensions',
    'device.browser', 'device.deviceCategory', 'device.isMobile', 'device.operatingSystem',
    'geoNetwork.city', 'geoNetwork.continent', 'geoNetwork.country', 'geoNetwork.metro', 'geoNetwork.networkDomain',
    'geoNetwork.region', 'geoNetwork.subContinent',
    'totals.bounces', 'totals.hits', 'totals.newVisits', 'totals.pageviews', 'totals.transactionRevenue',
    'trafficSource.adContent', 'trafficSource.campaign', 'trafficSource.isTrueDirect', 'trafficSource.keyword',
    'trafficSource.medium', 'trafficSource.referralPath', 'trafficSource.source'
]

train_start = pd.to_datetime('20170501')
train_end = pd.to_datetime('20171015')
valid_start = pd.to_datetime('20171201')
valid_end = pd.to_datetime('20180131')
test_start = pd.to_datetime('20180501')

In [215]:
def load_df(csv_path):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    full_df = pd.DataFrame()
    dfs = pd.read_csv(csv_path, sep=',',
            converters={column: json.loads for column in JSON_COLUMNS}, 
            dtype={'fullVisitorId': 'str', 'sessionId': 'str'},
            chunksize=100000)
    for df in dfs:
        df.reset_index(drop=True, inplace=True)
        for column in JSON_COLUMNS:
            column_as_df = json_normalize(df[column])
            column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
            df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)

        use_df = df[features]
        del df
        gc.collect()
        full_df = pd.concat([full_df, use_df], axis=0).reset_index(drop=True)
    return full_df

def mark_dataset(d):
    if train_start <= d <= train_end:
        return 'train'
    elif valid_start <= d <= valid_end:
        return 'valid'
    return 'none'

def mark_weeks(date_string, start):
    return (pd.to_datetime(date_string) - start).days // 7

def mark_months(date_string, start):
    return (pd.to_datetime(date_string) - start).days // 30

def feature_gen(df, agg_col=None):
    if agg_col is not None:
        gp_col = ['fullVisitorId', agg_col]
    else:
        gp_col = 'fullVisitorId'
    X = df.groupby(gp_col).agg({
        'channelGrouping': {
            'num_unique_channels': lambda x: len(set(x)),
            'most_freq_channel': lambda x: x.mode(),
        },
        'visitId': {
            'num_sessions': lambda x: len(set(x)),
        },
        'device.browser': {
            'num_unique_browsers': lambda x: len(set(x)),
            'most_freq_browser': lambda x: x.mode(),
        },
        'device.deviceCategory': {
            'num_unique_device': lambda x: len(set(x)),
            'most_freq_device': lambda x: x.mode(),        
        },
        'device.isMobile': {
            'mobile': lambda x: np.round(x.mean()),
        },
        'device.operatingSystem': {
            'num_unique_os': lambda x: len(set(x)),
            'most_freq_os': lambda x: x.mode(),        
        },
        'geoNetwork.city': {
            'num_unique_cities': lambda x: len(set(x)),
            'most_freq_city': lambda x: x.mode(),        
        },
        'geoNetwork.country': {
            'num_unique_countries': lambda x: len(set(x)),
            'most_freq_country': lambda x: x.mode(),        
        },
        'geoNetwork.continent': {
            'num_unique_continents': lambda x: len(set(x)),
            'most_freq_continent': lambda x: x.mode(),        
        },
        'geoNetwork.networkDomain': {
            'num_unique_domains': lambda x: len(set(x)),
            'most_freq_domain_suffix': lambda x: x.mode()[0].split('.')[-1],        
        },
        'totals.bounces': {
            'mean_bounces': lambda x: x.fillna(0).astype(int).mean(),
        },
        'totals.hits': {
            'mean_hits': lambda x: x.fillna(0).astype(int).mean(),
            'std_hits': lambda x: x.fillna(0).astype(int).std(),
            'max_hits': lambda x: x.fillna(0).astype(int).max(),
        },
        'totals.newVisits': {
            'num_newVisits': lambda x: x.fillna(0).astype(int).sum(),
        },
        'totals.pageviews': {
            'num_pageviews': lambda x: x.fillna(0).astype(int).sum(),
        },
        'totals.transactionRevenue': {
            'num_transactions': lambda x: len(set(x)),
            'total_revenue': lambda x: x.fillna(0).astype(int).sum()
        },
        'trafficSource.adContent': {
            'num_ads': lambda x: len(set(x)),
            'most_freq_ads': lambda x:  x.fillna('unknown').mode()[0],        
        },
        'trafficSource.campaign': {
            'num_camps': lambda x: len(set(x)),
            'most_freq_camp': lambda x: x.fillna('unknown').mode()[0],        
        },    
        'trafficSource.isTrueDirect': {
            'direct': lambda x: x.fillna(0).astype(int).mean(),
        },    
        'trafficSource.keyword': {
            'num_keywords': lambda x: len(set(x.fillna('unknown').map(lambda x: x.lower().split()[0]))),
            'most_freq_keywords': lambda x: x.fillna('unknown').map(lambda x: x.lower().split()[0]).mode()[0],
        },
        'trafficSource.medium': {
            'num_medium': lambda x: len(set(x)),
            'most_freq_medium': lambda x: x.fillna('unknown').mode()[0],        
        },
        'trafficSource.referralPath': {
            'num_ref': lambda x: len(set(x.fillna('/').map(lambda x: x.split('/')[1]))),
            'most_freq_ref': lambda x: x.fillna('/').map(lambda x: x.lower().split('/')[1]).mode()[0],     
        },
        'trafficSource.source': {
            'num_source': lambda x: len(set(x.fillna('.').map(lambda x: x.split('.')[0]))),
            'most_freq_source': lambda x: x.fillna('.').map(lambda x: x.lower().split('.')[0]).mode()[0],     
        }
    })
    X.reset_index(inplace=True)
    if agg_col is not None:
        X = X.pivot(index='fullVisitorId', columns=agg_col)
        X.reset_index(inplace=True)
    return X

def count_encoding(c):
    c = c.fillna('__unknown__').map(lambda x: ''.join(x))
    c_maps = c.value_counts().to_dict()
    c_mapped = c.map(c_maps)
    return c_mapped

def duplicate_columns(df, return_dataframe = False, verbose = False):
    '''
        a function to detect and possibly remove duplicated columns for a pandas dataframe
    '''
    # group columns by dtypes, only the columns of the same dtypes can be duplicate of each other
    groups = df.columns.to_series().groupby(df.dtypes).groups
    duplicated_columns = []
 
    for dtype, col_names in groups.items():
        column_values = df[col_names]
        num_columns = len(col_names)
 
        # find duplicated columns by checking pairs of columns, store first column name if duplicate exist 
        for i in range(num_columns):
            column_i = column_values.iloc[:,i].values
            for j in range(i + 1, num_columns):
                column_j = column_values.iloc[:,j].values
                if np.array_equal(column_i, column_j):
                    if verbose: 
                        print("column {} is a duplicate of column {}".format(col_names[i], col_names[j]))
                    duplicated_columns.append(col_names[i])
                    break
    if not return_dataframe:
        # return the column names of those duplicated exists
        return duplicated_columns
    else:
        # return a dataframe with duplicated columns dropped 
        return df.drop(labels = duplicated_columns, axis = 1)

In [4]:
%time data = load_df('../data/train_v2.csv')

CPU times: user 7min 54s, sys: 17.3 s, total: 8min 12s
Wall time: 7min 1s


In [5]:
%time test = load_df('../data/test_v2.csv')

CPU times: user 1min 52s, sys: 3.43 s, total: 1min 56s
Wall time: 1min 40s


In [6]:
sub = pd.read_csv('../data/sample_submission_v2.csv', dtype={'fullVisitorId': 'str'})

In [7]:
data.date = pd.to_datetime(data.date.astype('str'))
test.date = pd.to_datetime(test.date.astype('str'))

In [8]:
print(data.date.min(), data.date.max())
print(test.date.min(), test.date.max())

2016-08-01 00:00:00 2018-04-30 00:00:00
2018-05-01 00:00:00 2018-10-15 00:00:00


In [9]:
%time data['subset'] = data['date'].map(mark_dataset)

CPU times: user 4.72 s, sys: 60.8 ms, total: 4.78 s
Wall time: 4.2 s


In [10]:
train = data.loc[data.subset == 'train']
valid = data.loc[data.subset == 'valid']

In [11]:
del data
gc.collect()

7

In [12]:
valid['totals.transactionRevenue'] = valid['totals.transactionRevenue'].fillna(0)
valid['totals.transactionRevenue'] = valid['totals.transactionRevenue'].astype(int)
targets = pd.DataFrame({
    'fullVisitorId': train.fullVisitorId.unique(),
    'totals.transactionRevenue': 0
})
valid_targets = valid.groupby('fullVisitorId')['totals.transactionRevenue'].agg(sum).reset_index()
targets = targets.merge(right=valid_targets, how='left', on='fullVisitorId', suffixes=['', '_new'])
targets['totals.transactionRevenue_new'] = targets['totals.transactionRevenue_new'].fillna(0)
targets['totals.transactionRevenue_new'] = targets['totals.transactionRevenue_new'].astype(int)
targets['target'] = targets['totals.transactionRevenue'] + targets['totals.transactionRevenue_new']
targets = targets[['fullVisitorId', 'target']]

In [14]:
train['week'] = train['date'].map(partial(mark_weeks, start=train_start))
test['week'] = test['date'].map(partial(mark_weeks, start=test_start))
train['month'] = train['date'].map(partial(mark_months, start=train_start))
test['month'] = test['date'].map(partial(mark_months, start=test_start))

In [33]:
test['trafficSource.referralPath'] = test['trafficSource.referralPath'].map(lambda x: x if x != '(not set)' else '/')

In [18]:
%time X_train_week = feature_gen(train, agg_col='week')

/home/ren/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


CPU times: user 1h 7min 17s, sys: 31.5 s, total: 1h 7min 49s
Wall time: 1h 5min 50s


In [19]:
%time X_train_month = feature_gen(train, agg_col='month')

/home/ren/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


CPU times: user 1h 4min 45s, sys: 28.2 s, total: 1h 5min 14s
Wall time: 1h 3min 20s


In [45]:
%time X_train_whole = feature_gen(train, agg_col=None)

/home/ren/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


CPU times: user 1h 4min, sys: 24.3 s, total: 1h 4min 24s
Wall time: 1h 3min 14s


In [36]:
%time X_test_week = feature_gen(test, agg_col='week')

/home/ren/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


CPU times: user 49min 22s, sys: 24.6 s, total: 49min 47s
Wall time: 48min 12s


In [37]:
%time X_test_month = feature_gen(test, agg_col='month')

/home/ren/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


CPU times: user 46min 24s, sys: 23.9 s, total: 46min 47s
Wall time: 45min 23s


In [46]:
%time X_test_whole = feature_gen(test, agg_col=None)

/home/ren/anaconda3/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:4658: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


CPU times: user 45min 58s, sys: 20.5 s, total: 46min 19s
Wall time: 45min 14s


In [51]:
print(X_train_week.shape[0] == X_train_month.shape[0] == X_train_whole.shape[0])
print(X_test_week.shape[0] == X_test_month.shape[0] == X_test_whole.shape[0])

True
True


In [72]:
print(all(X_train_week['fullVisitorId'] == X_train_month['fullVisitorId']))
print(all(X_train_week['fullVisitorId'] == X_train_whole['fullVisitorId']))
print(all(X_test_week['fullVisitorId'] == X_test_month['fullVisitorId']))
print(all(X_test_week['fullVisitorId'] == X_test_whole['fullVisitorId']))

True
True
True
True


In [101]:
%time X_train = pd.concat([X_train_week, X_train_month.iloc[:,1:], X_train_whole.iloc[:,1:]], axis=1, ignore_index=True)

CPU times: user 12.1 s, sys: 1.02 s, total: 13.2 s
Wall time: 5.39 s


In [102]:
%time X_test = pd.concat([X_test_week, X_test_month.iloc[:, 1:], X_test_whole.iloc[:, 1:]], axis=1, ignore_index=True)

CPU times: user 11.5 s, sys: 744 ms, total: 12.3 s
Wall time: 4.79 s


In [171]:
del X_train_week, X_train_month, X_train_whole
del X_test_week, X_test_month, X_test_whole
gc.collect()

1728

In [105]:
X_train.columns = ['fullVisitorId'] + ['fe' + str(i+1) for i in range(1210)]

In [106]:
X_test.columns = ['fullVisitorId'] + ['fe' + str(i+1) for i in range(1210)]

In [109]:
all(targets.fullVisitorId.isin(X_train.fullVisitorId))

True

In [110]:
X_train = X_train.merge(right=targets, how='left', on='fullVisitorId')

In [112]:
X_test['target'] = np.nan

In [113]:
print(X_test.shape)
print(X_train.shape)

(296530, 1212)
(329636, 1212)


In [116]:
X_train.to_csv('../data/processed_train.csv', index=False)
X_test.to_csv('../data/processed_test.csv', index=False)

In [185]:
X = pd.concat([X_train, X_test], axis=0)

In [186]:
X.shape

(626166, 1212)

In [187]:
X.dtypes.value_counts()

float64    725
object     466
int64       20
bool         1
dtype: int64

In [188]:
for idx, col in tqdm(enumerate(X.columns)):
    if col in  ['fullVisitorId', 'target']:
        continue
    if X[col].dtypes in ['object', 'bool']:
        X[col] = count_encoding(X[col].astype('str'))
#     elif X[col].dtypes == 'bool':
#         X[col] = count_encoding(X[col].astype('str'))
    else:
        X[col] = X[col].fillna(0)




0it [00:00, ?it/s]


26it [00:01, 22.71it/s]


27it [00:01, 14.70it/s]


28it [00:02, 10.71it/s]


29it [00:03,  8.85it/s]


30it [00:03,  7.54it/s]


31it [00:04,  6.68it/s]


32it [00:05,  5.95it/s]


33it [00:06,  5.48it/s]


34it [00:06,  5.04it/s]


35it [00:07,  4.72it/s]


36it [00:08,  4.44it/s]


37it [00:08,  4.24it/s]


38it [00:09,  4.04it/s]


39it [00:10,  3.90it/s]


40it [00:10,  3.77it/s]


41it [00:11,  3.65it/s]


42it [00:11,  3.55it/s]


43it [00:12,  3.45it/s]


44it [00:13,  3.35it/s]


45it [00:13,  3.29it/s]


46it [00:14,  3.23it/s]


47it [00:14,  3.16it/s]


48it [00:15,  3.12it/s]


49it [00:15,  3.07it/s]


77it [00:16,  4.80it/s]


98it [00:17,  5.76it/s]


105it [00:22,  4.67it/s]


110it [00:25,  4.27it/s]


114it [00:28,  4.01it/s]


117it [00:30,  3.87it/s]


119it [00:31,  3.78it/s]


121it [00:32,  3.71it/s]


146it [00:33,  4.31it/s]


148it [00:36,  4.11it/s]


149it [00:37,  4.02it/s]


150it [00:38,  3.93it/s]


151it [00:39,  3.86it/s]


152

In [189]:
X.loc[~X.target.isnull()].to_csv('../data/encoded_train.csv', index=False)
X.loc[X.target.isnull()].to_csv('../data/encoded_test.csv', index=False)

In [219]:
dup_indices = duplicate_columns(X.loc[~X.target.isnull()], return_dataframe=False, verbose=True)

column fe74 is a duplicate of column fe122
column fe75 is a duplicate of column fe123
column fe76 is a duplicate of column fe124
column fe77 is a duplicate of column fe125
column fe78 is a duplicate of column fe126
column fe79 is a duplicate of column fe127
column fe80 is a duplicate of column fe128
column fe82 is a duplicate of column fe130
column fe85 is a duplicate of column fe133
column fe86 is a duplicate of column fe134
column fe89 is a duplicate of column fe137
column fe93 is a duplicate of column fe141
column fe96 is a duplicate of column fe144
column fe122 is a duplicate of column fe194
column fe127 is a duplicate of column fe199
column fe128 is a duplicate of column fe200
column fe140 is a duplicate of column fe212


In [220]:
X.drop(labels = dup_indices, axis = 1, inplace=True)

In [221]:
X.shape

(626166, 1195)

In [222]:
X.loc[~X.target.isnull()].to_csv('../data/de_dup_encoded_train.csv', index=False)
X.loc[X.target.isnull()].to_csv('../data/de_dup_encoded_test.csv', index=False)

In [224]:
X.loc[~X.target.isnull()].groupby('fe1')['target'].mea

0.0    597118
1.0     28511
2.0       525
3.0        12
Name: fe1, dtype: int64